# Agent Backed by Azure OpenAI Assistant API

### Create a OpenAI Assistant in Autogen

In [2]:
import os

from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen import UserProxyAgent
from dotenv import load_dotenv

load_dotenv()


user_proxy = UserProxyAgent(
    "user_proxy", 
    max_consecutive_auto_reply=0,
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    }
)

assistant_id = os.environ.get("ASSISTANT_ID", None)
llm_config = {
    "config_list": [
    {
        "model": os.environ.get("OPENAI_GPT_DEPLOYMENT", None),
        "api_key": os.environ.get("OPENAI_KEY", None),
        "base_url": os.environ.get("OPENAI_URI", None),
        "api_type": "azure",
        "api_version": os.environ.get("OPENAI_VERSION", None),
    }
],
}
assistant_config = {
    # define the openai assistant behavior as you need
}
oai_agent = GPTAssistantAgent(
    name="oai_agent",
    instructions="I'm an openai assistant running in autogen",
    llm_config=llm_config,
    assistant_config=assistant_config,
)

Send a test question:

In [5]:
user_proxy.initiate_chat(recipient=oai_agent,
    message="Tell me a joke."
)

user_proxy (to oai_agent):

Tell me a joke.

--------------------------------------------------------------------------------
oai_agent (to user_proxy):

Why don't scientists trust atoms?

Because they make up everything!


--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Tell me a joke.', 'role': 'assistant'}, {'content': "Why don't scientists trust atoms?\n\nBecause they make up everything!\n", 'role': 'user'}], summary="Why don't scientists trust atoms?\n\nBecause they make up everything!\n", cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])


### Use OpenAI Assistant Built-in Tools and Function Calling

##### Code Interpreter

The Code Interpreter empowers your agents to write and execute Python code in a secure environment provide by OpenAI. This unlocks several capabilities, including but not limited to:

- Process data: Handle various data formats and manipulate data on the fly.
- Generate outputs: Create new data files or even visualizations like graphs.
- ...

Using the Code Interpreter with the following configuration.


In [6]:
assistant_config = {
    "tools": [
        {"type": "code_interpreter"},
    ]
}

code_oai_agent = GPTAssistantAgent(
    name="code_oai_agent",
    instructions="You are a code assistant tool that can run Python code",
    llm_config=llm_config,
    assistant_config=assistant_config,
)

In [7]:
user_proxy.initiate_chat(recipient=code_oai_agent,
    message="Run a Monty Hall problem simulation 10000 times, and tell me how often you win by switching doors."
)

user_proxy (to code_oai_agent):

Run a Monty Hall problem simulation 10000 times, and              tell me how often you win by switching doors.

--------------------------------------------------------------------------------
code_oai_agent (to user_proxy):

After simulating the Monty Hall problem 10,000 times and always switching doors, the contestant won approximately 66.44% of the time.


--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Run a Monty Hall problem simulation 10000 times, and              tell me how often you win by switching doors.', 'role': 'assistant'}, {'content': 'After simulating the Monty Hall problem 10,000 times and always switching doors, the contestant won approximately 66.44% of the time.\n', 'role': 'user'}], summary='After simulating the Monty Hall problem 10,000 times and always switching doors, the contestant won approximately 66.44% of the time.\n', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])



To get the file.id, you can employ two methods:

Azure OpenAI Playground: Leverage the Azure OpenAI Playground, an interactive platform accessible at https://oai.azure.com, to upload your files and obtain the corresponding file IDs.

Code-Based Uploading: Alternatively, you can upload files and retrieve their file IDs programmatically using the following code snippet:


In [16]:
from openai import AzureOpenAI

client = AzureOpenAI(
    base_url=os.environ.get("BASE_URL"),
    api_key=os.environ.get("OPENAI_KEY"),
    api_version=os.environ.get("OPENAI_VERSION")
)

file = client.files.create(
  file=open("mydata.csv", "rb"),
  purpose='assistants'
)



### File Search

The File Search tool empowers your agents to tap into knowledge beyond its pre-trained model. This allows you to incorporate your own documents and data, such as product information or code files, into your agent's capabilities.

Using the File Search with the following configuration.


OpenAI client config of GPTAssistantAgent(files_oai_agent) - model: gpt-4-1106-preview
No matching assistant found, creating a new assistant



Here's how to obtain the vector_store.id using two methods:

OpenAI Playground: Leverage the OpenAI Playground, an interactive platform accessible at https://platform.openai.com/playground, to create a vector store, upload your files, and add it into your vector store. Once complete, you'll be able to retrieve the associated vector_store.id.

Code-Based Uploading:Alternatively, you can upload files and retrieve their file IDs programmatically using the following code snippet:


In [2]:
from openai import AzureOpenAI

client = AzureOpenAI(
    base_url=os.environ.get("BASE_URL"),
    api_key=os.environ.get("OPENAI_KEY"),
    api_version=os.environ.get("OPENAI_VERSION")
)

# Step 1: Create a Vector Store
vector_store = client.beta.vector_stores.create(name="Financial Statements")
print("Vector Store created:", vector_store.id)  # This is your vector_store.id

# Step 2: Prepare Files for Upload
file_paths = ["./Gotcha-Covered-Design-Style-Guide.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

# Step 3: Upload Files and Add to Vector Store (with status polling)
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# Step 4: Verify Completion (Optional)
print("File batch status:", file_batch.status)
print("Uploaded file count:", file_batch.file_counts.completed)


Vector Store created: vs_djfk94WgbkQNpXrfvy0xDBlF
File batch status: completed


AttributeError: 'FileCounts' object has no attribute 'processed'

In [3]:
assistant_config = {
    "tools": [
        {"type": "retrieval"}
    ],
    "tool_resources": {
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    }
}

files_oai_agent = GPTAssistantAgent(
    name="files_oai_agent",
    instructions="You are a files search assistant tool",
    llm_config=llm_config,
    assistant_config=assistant_config
)


OpenAI client config of GPTAssistantAgent(files_oai_agent) - model: gpt-4-1106-preview


No matching assistant found, creating a new assistant


In [7]:
user_proxy.initiate_chat(recipient=files_oai_agent,
    message="What's a modern urban style like?"
)

INFO:autogen.agentchat.contrib.gpt_assistant_agent:Clearing thread thread_36qXzpJmgJhAcA20CmPEHSTL


user_proxy (to files_oai_agent):

What's a modern urban style like?

--------------------------------------------------------------------------------
files_oai_agent (to user_proxy):

Modern urban design style is characterized by simplicity, sleekness, and a minimalist approach. It showcases true form and function, featuring cleaner lines and open floor plans. Originating from old factories and industrial spaces, modern urban design employs stripped-back architecture and elements like salvaged or repurposed items. It is inspired by fuss-free approaches using basic materials such as untreated timber beams, brick, reinforced concrete, and exposed metal structures. The design style incorporates raw materials and builds around the structure of the existing space, often with strong horizontal and vertical lines and a simple, neutral color palette accented by a bold color. Metallic hardware like stainless steel or nickel is common, with a "less is more" approach to accessories and decoration

ChatResult(chat_id=None, chat_history=[{'content': "What's a modern urban style like?", 'role': 'assistant'}, {'content': 'Modern urban design style is characterized by simplicity, sleekness, and a minimalist approach. It showcases true form and function, featuring cleaner lines and open floor plans. Originating from old factories and industrial spaces, modern urban design employs stripped-back architecture and elements like salvaged or repurposed items. It is inspired by fuss-free approaches using basic materials such as untreated timber beams, brick, reinforced concrete, and exposed metal structures. The design style incorporates raw materials and builds around the structure of the existing space, often with strong horizontal and vertical lines and a simple, neutral color palette accented by a bold color. Metallic hardware like stainless steel or nickel is common, with a "less is more" approach to accessories and decorations. Modern urban design favors large windows that let in plent

### Function calling

Function Calling empowers you to extend the capabilities of your agents with your pre-defined functionalities, which allows you to describe custom functions to the Assistant, enabling intelligent function selection and argument generation.

Using the Function calling with the following configuration.

# learn more from https://platform.openai.com/docs/guides/function-calling/function-calling


In [9]:
from autogen.function_utils import get_function_schema

@user_proxy.register_for_execution()
def get_current_weather(location: str) -> dict:
    """
    Retrieves the current weather for a specified location.

    Args:
    location (str): The location to get the weather for.

    Returns:
    Union[str, dict]: A dictionary with weather details..
    """

    # Simulated response
    return {
        "location": location,
        "temperature": 22.5,
        "description": "Partly cloudy"
    }

api_schema = get_function_schema(
    get_current_weather,
    name=get_current_weather.__name__,
    description="Returns the current weather data for a specified location."
)

assistant_config = {
    "tools": [
        api_schema
    ],
}

functions_oai_agent = GPTAssistantAgent(
    name="functions_oai_agent",
    instructions="You are a weather assistant",
    llm_config=llm_config,
    assistant_config=assistant_config,
    function_map={
        "get_current_weather": get_current_weather
    }
)

user_proxy.initiate_chat(recipient=functions_oai_agent,
    message="What's the weather like in Redmond, WA?"
)

user_proxy (to functions_oai_agent):

What's the weather like in Redmond, WA?

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_current_weather...


INFO:autogen.agentchat.contrib.gpt_assistant_agent:Intermediate executing(get_current_weather, Success: True) : {'location': 'Redmond, WA', 'temperature': 22.5, 'description': 'Partly cloudy'}


functions_oai_agent (to user_proxy):

The current weather in Redmond, WA is partly cloudy with a temperature of 22.5°C.


--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': "What's the weather like in Redmond, WA?", 'role': 'assistant'}, {'content': 'The current weather in Redmond, WA is partly cloudy with a temperature of 22.5°C.\n', 'role': 'user'}], summary='The current weather in Redmond, WA is partly cloudy with a temperature of 22.5°C.\n', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])